In [5]:
import requests
import csv
import statistics
import time

res = requests.get("https://api.blockchain.info/charts/hash-rate?cors=true&format=csv&lang=de").text
lines = list(csv.reader(res.splitlines()))

# Average over the last 60 days and convert from THash/s to Hash/s
hashrate_60d_avg = statistics.mean(float(rate_s)*10e12 for _, rate_s in lines[-60:]) # Hash/s

* https://en.bitcoin.it/wiki/Mining_hardware_comparison#cite_note-AntMinerS9-9
* https://www.bitmaintech.com/productDetail.htm?pid=0002016052907243375530DcJIoK0654
* https://blockchain.info/api/blockchain_api

In [6]:

# Very optimistic lower bound, ignoring older hardware, cooling etc. pp.
power_consumption = 0.098e-9 # J/Hash

# 1J = 1Ws → Hash/s * J/Hash = J/s = Ws/s = W
network_power_consumption = hashrate_60d_avg * power_consumption # W

print('Total network power consumption (estimated lower bound):', network_power_consumption*1e-6, 'MW')

Total network power consumption (estimated lower bound): 1756.567393097864 MW


In [7]:
# Fetch blockchain.info data on the last [n] blocks
import sqlite3

db = sqlite3.connect('blockinfo.db')
db.execute('CREATE TABLE IF NOT EXISTS blockinfo (idx INTEGER, info TEXT)')

def fetch_blocks(n=100): # approx. 1k blocks in 7d
    latest = requests.get('https://blockchain.info/latestblock').json()['block_index']
    with db as conn:
        for idx in range(latest, latest-n, -1):
            if conn.execute('SELECT count(1) FROM blockinfo WHERE idx=?', (idx,)).fetchone()[0] == 0:
                print('fetching:', idx, flush=True)
                time.sleep(1)
                info = requests.get('https://blockchain.info/block-index/{}?format=json'.format(idx)).text
                conn.execute('INSERT INTO blockinfo VALUES (?, ?)', (idx, info))
#            else:
#                print('skipping:', idx, flush=True)
    print('done.')

def blocks(lower, upper, conn=db):
    for idx, in conn.execute('SELECT idx FROM blockinfo WHERE idx BETWEEN ? AND ? ORDER BY idx', (lower, upper)):
        yield json.loads(conn.execute('SELECT info FROM blockinfo WHERE idx=?', (idx,)).fetchone()[0])

In [8]:
#fetch_blocks(1000)

In [9]:
import json
import statistics
def avg_tx_last_n(n=1000):
    with db as conn:
        last_idx = db.execute('SELECT MAX(idx) FROM blockinfo').fetchone()[0]
        data = [
            (len(b['tx']), b['time']) for b in blocks(last_idx-n, last_idx, conn)
        ]
        avg_tx_per_block = statistics.mean(ntx for ntx, time in data)
        avg_seconds_per_block = (data[-1][1] - data[0][1])/n
    avg_tx_per_second = avg_tx_per_block / avg_seconds_per_block
    return avg_tx_per_second

avg_tx_last_1000 = avg_tx_last_n(n=1000)
avg_tx_last_1000

2.823744356770102

In [10]:
energy_per_tx = network_power_consumption * 1 / avg_tx_last_1000 # J
print('Total network energy consumption per transaction (estimated lower bound):', energy_per_tx*1e-6, 'MJ')

Total network energy consumption per transaction (estimated lower bound): 622.0702624465223 MJ


In [11]:
kcal_per_day_adult_male = 2600
J_per_day_adult_male = 180/43 * 1000 * kcal_per_day_adult_male
adult_males_nourished_1d_per_tx = energy_per_tx/J_per_day_adult_male
print('Days an adult male could be nourished from the energy a single transaction consumes (approximated, dependent on person):', adult_males_nourished_1d_per_tx)

Days an adult male could be nourished from the energy a single transaction consumes (approximated, dependent on person): 57.15602838718047


* https://en.wikipedia.org/wiki/Calorie
* https://en.wikipedia.org/wiki/Food_energy 

In [12]:
bn_info = requests.get('https://bitnodes.21.co/api/v1/snapshots/latest/').json()

In [31]:
import collections
countries = collections.Counter()
total_nodes = bn_info['total_nodes']
for pver, uagent, conntime, svcs, height, host, city, country, lat, lon, tz, asn, org in bn_info['nodes'].values():
    countries[country] += 1
country_fractions = [ (country, count/total_nodes) for country, count in countries.most_common() ]
cf_idx = 10
print(country_fractions[:cf_idx], sum(frac for country, frac in country_fractions[:cf_idx]))

[('US', 0.26956361627692876), ('DE', 0.17142331062419444), ('FR', 0.08322592524396981), ('NL', 0.056711471183944026), ('CA', 0.04419075676670963), ('GB', 0.04363837230712576), ('CN', 0.03387958018781072), (None, 0.033511323881421466), ('RU', 0.027435094825998894), ('SE', 0.017308046400294604)] 0.7808874976983979


* https://bitnodes.21.co/api/

In [14]:
import csv

iso_3166_data = requests.get('http://www.iso.org/iso/home/standards/country_codes/country_names_and_code_elements_txt-temp.htm')
iso_3166_data = iso_3166_data.text.splitlines()[1:-1]
iso_3166_1_alpha_2_to_name = {
    cc.lower(): name.lower() for name, cc in ( line.split(';') for line in iso_3166_data )
}

* https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2

In [58]:
from bs4 import BeautifulSoup

#energy_by_coutry_xls = requests.get('http://www.iea.org/media/statistics/surveys/electricity/mes.xls').content
energy_by_coutry_table = requests.get('http://wdi.worldbank.org/AjaxDownload/FileDownloadHandler.ashx?filename=3.7_Electricity_production_sources_and_access.xls&filetype=excel').content
# don't ask why, but the above link actually returns readable html. And who am I to complain that I get HTML to parse instead of an Excel file? =)
soup = BeautifulSoup(energy_by_coutry_table, 'lxml')
tbody = soup.find('tbody') # data table body
energy_by_country = {
    cname.text.lower(): [ float(col.text.replace(',', ''))/100 if col.text != '..' else None for col in numbers ]
            for cname, *numbers in ( row.find_all('td') for row in tbody.find_all('tr') )
}

* http://wdi.worldbank.org/table/3.7#

In [78]:
import numpy as np
from IPython.display import display, HTML

fracs = np.zeros(6, dtype=np.float) # coal, gas, oil, hydro, renewable, nuclear
accounted_total = 0
for cc, frac_total in country_fractions:
    if cc is None:
        break
    twh_total, *cfracs, access = energy_by_country[iso_3166_1_alpha_2_to_name[cc.lower()]]
    accounted_total += frac_total
    fracs += np.array(cfracs)*frac_total
fracs /= accounted_total

In [79]:
display(HTML('<table><tr><th>Total</th><th>Coal</th><th>Gas</th><th>Oil</th><th>Hyrdo</th><th>Renewable</th><th>Nuclear</th></tr>'
        '<tr><td>{:.02f}</td><td>'.format(accounted_total*100) +\
        '</td><td>'.join('{:.02f}'.format(x*100) for x in fracs) + '</td></tr></table>'))

Total,Coal,Gas,Oil,Hyrdo,Renewable,Nuclear
70.26,36.01,20.21,0.88,9.47,10.15,22.55
